In [ ]:
import cv2
import numpy as np
import os
#Clean anh 
def clean_images():
    
    file_list=os.listdir('./')
    for file_name in file_list:
        if '.png' in file_name:
            os.remove(file_name)

#Ham xu ly anh
def preprocess_image(image):
    
    #Chuyen anh sang he mau HSV
    hsv=cv2.cvtColor(image,cv2.COLOR_BGR2HSV)

    #Mau do (red color)
    lower_red1=np.array([0,150,50])
    upper_red1=np.array([10,255,150])
    lower_red2=np.array([150,100,20])
    upper_red2=np.array([180,255,150])
    mask_red1=cv2.inRange(hsv,lower_red1,upper_red1)
    mask_red2=cv2.inRange(hsv,lower_red2,upper_red2)
    mask_red=cv2.bitwise_or(mask_red1,mask_red2)

    #Mau xanh da troi (blue color)
    lower_blue=np.array([105,100,120])
    upper_blue=np.array([110,255,255])
    mask_blue=cv2.inRange(hsv,lower_blue,upper_blue)

    #Lam min va xu ly mat na
    mask_red=cv2.GaussianBlur(mask_red,(5,5),0)
    mask_red=cv2.erode(mask_red,None,iterations=2)
    mask_red=cv2.dilate(mask_red,None,iterations=2)

    mask_blue=cv2.GaussianBlur(mask_blue,(5,5),0)
    mask_blue=cv2.erode(mask_blue,None,iterations=2)
    mask_blue=cv2.dilate(mask_blue,None,iterations=2)

    return mask_red,mask_blue

def detect_circle_red(contour):
    
    area=cv2.contourArea(contour)
    
    #Loai tru cac hinh tron nho, tranh phat hien sai
    if area<500:
        return False

    perimeter=cv2.arcLength(contour,True)
    
    if perimeter == 0:
        return False

    x,y,w,h=cv2.boundingRect(contour)
    aspect_ratio=float(w)/h
    circularity=(4*np.pi*area)/(perimeter**2)
    
    #Danh sach dieu kien
    dks=[
        {
            "circularity_range": (0.65,1),
            "aspect_ratio_range": (0.75,1.2),
            "height_range": (37,120),
            "perimeter_min": 175
        },
        {
            "circularity_range": (0.23,0.24),
            "aspect_ratio_range": (0.75,1.2),
            "height_range": (37,120),
            "area_range": (595,700),
            "perimeter_range": (165,180)
        },
        {
            "circularity_range": (0.12,0.13),
            "area_range": (1590,1658),
            "perimeter_range": (390,420),
        }
    ]

    #kemtra
    for dk in dks:
        if (dk.get("circularity_range",(0,1))[0]<=circularity<=dk.get("circularity_range",(0,1))[1] and
            dk.get("aspect_ratio_range",(0,float("inf")))[0]<=aspect_ratio<=dk.get("aspect_ratio_range",(0,float("inf")))[1] and
            dk.get("height_range",(0,float("inf")))[0]<h<dk.get("height_range",(0,float("inf")))[1] and
            perimeter>=dk.get("perimeter_min",0) and
            dk.get("area_range",(0,float("inf")))[0]<=area<=dk.get("area_range",(0,float("inf")))[1] and
            dk.get("perimeter_range",(0,float("inf")))[0]<=perimeter<=dk.get("perimeter_range",(0,float("inf")))[1]):
            return True

    return False



#Ham phat hien hinh tron mau xanh
def detect_circle_blue(contour):
    area=cv2.contourArea(contour)
    if area<2300:
        return False

    perimeter=cv2.arcLength(contour,True)
    if perimeter == 0:
        return False

    x,y,w,h=cv2.boundingRect(contour)
    aspect_ratio=float(w)/h
    circularity=(4*np.pi*area)/(perimeter**2)

    #Dieu kien cho cac loai hinh tron khac nhau
    small_circle=0.67<=circularity<=1 and 0.9<=aspect_ratio<=1.2 and 37<h<150
    medium_circle=0.36<=circularity<0.67 and 0.9<=aspect_ratio<=1.2 and 37<h<150 and area>8500 and perimeter>500
    large_circle=0.25<=circularity<0.36 and 0.9<=aspect_ratio<=1.2 and 37<h<150 and area>14500 and perimeter>700

    #Dieu kien loai tru
    exclusion_area_perimeter=area<2500 and perimeter<210

    #Kiem tra cac dieu kien
    if exclusion_area_perimeter:
        return False
    if small_circle or medium_circle or large_circle:
        return True

    return False



def detect_triangle_red(contour):
    
    area=cv2.contourArea(contour)
    
    if area<2300:
        return False

    perimeter=cv2.arcLength(contour,True)
    approx=cv2.approxPolyDP(contour,0.04*perimeter,True)

    if len(approx) == 3:  #Hinh tam giac co 3 canh
        
        x,y,w,h=cv2.boundingRect(approx)
        aspect_ratio=float(w)/h

        #Dieu kien loai tru cho hinh tam giac nho
        if area<1400 and perimeter<150:
            return False

        #Dieu kien cho hinh tam giac hop le
        valid_triangle=0.9<aspect_ratio<1 and 30<w<150 and 30<h<150
        
        if valid_triangle:
            return True

    return False


def detect_rectangle_blue(contour):
    
    area=cv2.contourArea(contour)
    
    if area<1700:
        return False

    perimeter=cv2.arcLength(contour,True)
    approx=cv2.approxPolyDP(contour,0.02*perimeter,True)

    if len(approx)!=4:
        return False

    x,y,w,h=cv2.boundingRect(approx)
    aspect_ratio=float(w)/h

    #Cac dieu kien kich thuoc cho hinh chu nhat mong muon
    large_rectangle=w<150 and area>19000
    medium_rectangle=44<w<90 and 32<h<60 and 1200<area<6000 and 140<perimeter<400
    unwanted_rectangle=95<w<153 and 50<h<86 and perimeter<460 and area<8300

    #Loai tru cac truong hop chu vi va dien tich khong phu hop
    high_perimeter_exclusion=perimeter>700 and area<10000
    low_area_exclusion=perimeter>100 and area<900

    #Cac dieu kien ty le khung hinh va kich thuoc
    small_aspect_ratio=0.9<aspect_ratio<2 and 20<w<90 and 20<h<185
    large_aspect_ratio=0.9<aspect_ratio<2 and 90<w<300 and 20<h<185

    #Kiem tra cac dieu kien loai tru va mong muon
    if large_rectangle or medium_rectangle:
        return True
    
    if unwanted_rectangle or high_perimeter_exclusion or low_area_exclusion:
        return False
    
    if small_aspect_ratio or large_aspect_ratio:
        return True

    return False

#Kiem tra cac bien bao
def detect_traffic_signs(image):
    mask_red, mask_blue = preprocess_image(image)
    contours_red, _ = cv2.findContours(mask_red, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours_blue, _ = cv2.findContours(mask_blue, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    traffic_signs = []
    red_bboxes = []

    # Process red shapes (circle and triangle)
    for contour in contours_red:
        if detect_circle_red(contour):
            x, y, w, h = cv2.boundingRect(contour)
            roi = image[y:y+h, x:x+w]  # Extract the region of interest
            label = predict_traffic_sign(roi)  # Predict the traffic sign label
            red_bboxes.append((x, y, w, h))
            traffic_signs.append(('Red Circle', x, y, w, h, traffic_sign_labels.get(label, 'Unknown')))
            
        elif detect_triangle_red(contour):
            x, y, w, h = cv2.boundingRect(contour)
            roi = image[y:y+h, x:x+w]  # Extract the region of interest
            label = predict_traffic_sign(roi)  # Predict the traffic sign label
            red_bboxes.append((x, y, w, h))
            traffic_signs.append(('Red Triangle', x, y, w, h, traffic_sign_labels.get(label, 'Unknown')))

    # Process blue shapes (circle, rectangle)
    for contour in contours_blue:
        x, y, w, h = cv2.boundingRect(contour)
        is_inside_red_area = any(rx <= x <= rx + rw and ry <= y <= ry + rh for rx, ry, rw, rh in red_bboxes)

        if not is_inside_red_area:
            roi = image[y:y+h, x:x+w]  # Extract the region of interest
            label = predict_traffic_sign(roi)  # Predict the traffic sign label
            area = cv2.contourArea(contour)
            perimeter = cv2.arcLength(contour, True)
            circularity = (4 * np.pi * area) / (perimeter ** 2)

            if detect_circle_blue(contour):
                traffic_signs.append(('Blue Circle', x, y, w, h, traffic_sign_labels.get(label, 'Unknown')))
            elif detect_rectangle_blue(contour):
                traffic_signs.append(('Blue Rectangle', x, y, w, h, traffic_sign_labels.get(label, 'Unknown')))

    return traffic_signs


# Define the model architecture
class TrafficSignModel:
    def __init__(self):
        # Tạo các trọng số ngẫu nhiên cho các lớp Conv và FC
        self.conv1_weights = np.random.randn(64, 3, 3, 3)  # 64 filters, 3 channels, 3x3 kernel
        self.conv2_weights = np.random.randn(128, 64, 3, 3)  # 128 filters, 64 input channels, 3x3 kernel
        self.fc1_weights = np.random.randn(128 * 16 * 16, 512)  # Fully connected layer
        self.fc2_weights = np.random.randn(512, 10)  # Output layer, 10 classes

    def relu(self, x):
        return np.maximum(0, x)

    def max_pool(self, x, pool_size=2):
        return x[::pool_size, ::pool_size]

    def conv2d(self, input, weights, stride=1, padding=1):
        # Hàm convolution đơn giản
        input_padded = np.pad(input, ((padding, padding), (padding, padding), (0, 0)), mode='constant')
        output_shape = ((input.shape[0] - weights.shape[2] + 2 * padding) // stride + 1,
                        (input.shape[1] - weights.shape[3] + 2 * padding) // stride + 1,
                        weights.shape[0])

        output = np.zeros(output_shape)

        for i in range(output_shape[0]):
            for j in range(output_shape[1]):
                for f in range(weights.shape[0]):
                    output[i, j, f] = np.sum(input_padded[i*stride:i*stride+weights.shape[2], 
                                                          j*stride:j*stride+weights.shape[3], 
                                                          :] * weights[f])

        return output

    def forward(self, x):
        # Áp dụng Conv1
        x = self.relu(self.conv2d(x, self.conv1_weights))
        x = self.max_pool(x)
        
        # Áp dụng Conv2
        x = self.relu(self.conv2d(x, self.conv2_weights))
        x = self.max_pool(x)

        # Flattens the output for the fully connected layers
        x = x.flatten()

        # Áp dụng FC1
        x = self.relu(np.dot(x, self.fc1_weights))

        # Áp dụng FC2
        x = np.dot(x, self.fc2_weights)

        return x

model = TrafficSignModel()

# Class labels (adjust this as per your model's training labels)
traffic_sign_labels = {
    0: 'Thang hoac Phai, cam queo Trai', 
    1: 'Cam di nguoc chieu', 
    2: 'Cam re trai',
    7: 'Canh bao co tre em', 
    8: 'Cam dau xe', 
    4: 'Di cham thoi', 
    9: 'Cam dung va do xe', 
    6: 'Huong di theo vach ke duong', 
    3: 'Gap khuc phai', 
    5: 'Huong phai di vung phai', 
}

def predict_traffic_sign(model, image_path):
    image = preprocess_image(image_path)
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    output = model.forward(image)
    predicted_class = np.argmax(output)
    return predicted_class


# Load the pre-trained model
model = TrafficSignModel()
model.load_state_dict('traffic_sign_model.pth')
model.eval()  # Set the model to evaluation mode

def draw_and_save_traffic_signs(input_video_path, output_video_path='output.avi', delay=10):
    cap = cv2.VideoCapture(input_video_path)

    if not cap.isOpened():
        print("Error: Cannot open video file.")
        return

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_rate = int(cap.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (frame_width, frame_height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break  # End of video

        # Detect traffic signs
        traffic_signs = detect_traffic_signs(frame)

        # Loop through each detected traffic sign
        for sign_type, x, y, w, h, label in traffic_signs:
            color = (0, 255, 0) if 'Blue' in sign_type else (0, 0, 255)  
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

            label_text = label
            text_color = (255, 255, 255)
            cv2.putText(frame, label_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, text_color, 2, cv2.LINE_AA)

        # Save the frame to the output video
        out.write(frame)

        # No need to display the frame, just save it to video

        if cv2.waitKey(delay) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Example usage
input_video_path = 'video1.mp4'  # Your input video
output_video_path = 'output_video_2711.avi'  # Output video file
draw_and_save_traffic_signs(input_video_path, output_video_path)



IndentationError: expected an indented block after function definition on line 318 (3081764887.py, line 320)